In [6]:
from sclibrary.network_reader import NetworkReader
from sclibrary.simplicial_complex import SimplicialComplexNetwork
from sclibrary.plot import SCPlot 
import networkx as nx

data_folder = 'data/paper_data'

# read csv
filename = data_folder + '/edges.csv'
delimeter = " "
src_col = 'Source'
dest_col = 'Target'
feature_cols = ['Distance']

G = NetworkReader.read_csv(filename=filename, delimeter=delimeter, src_col=src_col, dest_col=dest_col, feature_cols=feature_cols)
simplices = G.simplicies(condition="distance", dist_col_name="Distance", dist_threshold=1.5)

# if coordinates exist
filename = data_folder + '/coordinates.csv'
coordinates = NetworkReader.get_coordinates(filename=filename, 
                                            node_id_col="Id", 
                                            x_col='X', 
                                            y_col='Y', 
                                            delimeter=" ")

# create sc
sc = SimplicialComplexNetwork(simplices=simplices)
x = sc.lower_laplacian_matrix(rank=1)
x

array([[ 2.,  1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  2.,  1.,  1., -1., -1.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  2.,  0.,  1.,  0., -1.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  2., -1., -1.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  1., -1.,  2.,  1., -1.,  0.,  0.,  0.],
       [ 0., -1.,  0., -1.,  1.,  2.,  0.,  1.,  0., -1.],
       [ 0.,  0., -1.,  0., -1.,  0.,  2., -1., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  2.,  1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  2.,  1.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0., -1.,  1.,  2.]], dtype=float32)